In [ ]:
import cv2, threading 
class VIDEO():# image cache
    def __init__(self): 
        self.condition = threading.Condition() 
        self.frame = b''
        self.cap = cv2.VideoCapture("tcp://127.0.0.1:2222")  
        
    def updater(self):   
        while(self.cap.isOpened()):   
            with self.condition:
                ret, frame = self.cap.read() 
                ret, jpeg  = cv2.imencode('.jpg', frame)
                self.frame = jpeg.tobytes()
                self.condition.notify_all() 
video = VIDEO()
video_link_thread = threading.Thread(target=video.updater)
video_link_thread.start() 

In [ ]:
import flask 
app = flask.Flask(__name__)
print('Click Here: http://127.0.0.1:9999/stream.mjpg\n\n\n')
@app.route("/stream.mjpg") 
def mjpg():  
    def generator():    
        while True:   
            with video.condition:
                video.condition.wait()
                frame = video.frame  
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

@app.route('/')
def hello_world():
    return 'Hello, World!'

app.run('127.0.0.1', 9999)